# **Importing the required files**

In [115]:
# Standard library imports
from os import listdir                    # To list files in a directory
from os.path import isfile, join         # To check if a path is a file and to join paths
import os                                # General OS-level operations
import datetime                          # For working with dates and times

# Third-party library imports
from openpyxl import load_workbook       # To load and work with Excel .xlsx files
import pandas as pd                      # To handle tabular data and dataframes
import matplotlib.pyplot as plt          # To plot and visualize data
import numpy as np                       # To perform numerical operations
from google.colab import drive           # To access Google Drive in Google Colab environment
from datetime import datetime            # For working with dates and time
from dateutil.relativedelta import relativedelta # For working with dates and time


# **Connecting Our Drive To Retrieve The Required Files**

In [116]:
# Mount your Google Drive into the Colab environment
drive.mount('/content/drive')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/Stock Products'

# **Loading The Required File**

In [117]:
# List of country codes corresponding to folder names
list_countries = ["USA", "UK", "CA", "EU", "DE", "FR", "IT", "ES", "JP"]

# Initialize an empty DataFrame to collect all data
df = pd.DataFrame()

# Loop through each country folder
for i in list_countries:
    try:
        # Define root path to the Stock Products folder for the current country
        root_path = r"/content/drive/My Drive/Stock Products"
        root_path = os.path.join(root_path, i)  # append country folder name

        # Change working directory to the country folder
        os.chdir(root_path)

        # Get a list of files in the current country folder
        onlyfiles = [
            f for f in listdir(root_path)
            if isfile(join(root_path, f))  # include only files, not directories
        ]

        # Process each file in the folder
        for f in onlyfiles:
            # Read Excel file into a temporary DataFrame
            df_t = pd.read_excel(f)

            # Add metadata columns: Country, Month, Year
            df_t['Country'] = i
            df_t['Month'] = str(f).strip().split()[0]  # assumes filename starts with month
            df_t['Year'] = str(f).strip().split()[1][:4]  # assumes year comes after month

            # Clean column names: remove dashes and title-case them
            df_t.columns = (
                df_t.columns
                .str.replace('-', '', regex=False)
                .str.replace('–', '', regex=False)
                .str.title()
            )

            # Concatenate the current DataFrame with the main one
            df = pd.concat([df_t, df], ignore_index=True)

    except Exception:
        # Silently skip the country if an error occurs (e.g., folder missing, file corrupt)
        pass

>> We now have the **df** DataFrame containing the Amazon Seller Central data.

In [118]:
# 🔷 Define the root path to the folder where the file is stored
root_path = r'/content/drive/My Drive/Sales Data/Final Shape of Files'

# 🔷 Change the working directory to the root path
os.chdir(root_path)

# 🔷 Define the filename for the current price file
Current_price = 'Current_price'

# 🔷 Read the current price data into a DataFrame
df_cp = pd.read_csv(Current_price)

# 🔷 Sort by date descending and remove duplicate rows based on ASIN and Region
df_cp = df_cp.sort_values(by= 'Date', ascending = False).drop_duplicates(subset = ['ASIN', 'Region'])

# 🔷 Rename the column 'Region' to 'Country' for consistency
df_cp.rename(columns = {'Region': 'Country'}, inplace = True)

# 🔷 Replace specific country codes for better readability
df_cp['Country'].replace({'US':'USA', "GB": 'UK'}, inplace = True)

/tmp/ipython-input-3055462854.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cp['Country'].replace({'US':'USA', "GB": 'UK'}, inplace = True)


>> We now have a DataFrame named **df_cp**, which holds the most recently detected price for each of our products.

In [119]:
# 🔷 Define file name and columns to read
filname = 'Sales_full'
columns_to_read = ['SalesOrganic', 'SalesSponsoredProducts', 'SalesSponsoredDisplay', 'NetProfit', 'ASIN', 'Marketplace', 'Date']

# 🔷 Load the data
df_s = pd.read_csv(filname, usecols = columns_to_read)

# 🔷 Extract time-based features
df_s['Month'] = pd.to_datetime(df_s['Date'], format ="%d/%m/%Y").dt.month_name()
df_s['Year'] = pd.to_datetime(df_s['Date'], format ="%d/%m/%Y").dt.year

# 🔷 Handle missing values
df_s.fillna(0, inplace = True)

# 🔷 Calculate gross revenue
df_s['Gross Revenue'] = df_s['SalesOrganic'] +  df_s['SalesSponsoredProducts'] + df_s['SalesSponsoredDisplay']

# 🔷 Define mapping dictionary
country_to_marketplace = {
    'Amazon.com':'US',
    'Amazon.ca':'CA' ,
    'Amazon.co.uk':'UK',
    'Amazon.de':'DE',
    'Amazon.fr':'FR',
    'Amazon.it':'IT',
    'Amazon.es':'ES',
    'amazon.co.jp':'JP',
    'amazon.com.mx':'MX'
}

# 🔷 Map Marketplace to Country
df_s['Country'] = df_s['Marketplace'].replace(country_to_marketplace)

# 🔷 Change the value of the "US" for consistency
df_s['Country'].replace({'US':'USA'}, inplace = True)

/tmp/ipython-input-1186359108.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_s['Country'].replace({'US':'USA'}, inplace = True)


In [120]:
# 🔷 Prompt user for choice
mont_of_last_30days = input("Do you want to get the margin of the last month or specific month \nwrite the month like 'April'. Write anything else for the last 30 months for example n\n")

# 🔷 Define list of month names
months = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]


# 🔷 Get current date
now = datetime.now()

# 🔷 Calculate previous month
previous_month_date = now - relativedelta(months=1)

# 🔷 Get previous month as number and name
previous_month_num = previous_month_date.month
previous_month_name = previous_month_date.strftime("%B")

# 🔷 Filter data based on user choice
# 🔷 If the user chooses a specific month, df_s will contain the margins for that month
if mont_of_last_30days.lower().capitalize() in months:
    df_s= pd.pivot_table(df_s,index=['Year','Month','Country', 'ASIN'], values=([ 'Gross Revenue', 'NetProfit']), aggfunc = 'sum').reset_index()
    df_s = df_s[df_s['Month'] == mont_of_last_30days]

# 🔷 Otherwise, df_s will contain the margins for the previous month.
else:
    df_s= pd.pivot_table(df_s,index=['Year','Month','Country', 'ASIN'], values=([ 'Gross Revenue', 'NetProfit']), aggfunc = 'sum').reset_index()
    df_s = df_s[df_s['Month'] == previous_month_name]


# 🔷 Calculate Net Margin
df_s['Net Margin'] = df_s['NetProfit'] / df_s['Gross Revenue']

# 🔷 Clean up the data
df_s.fillna(0, inplace = True)
df_s.replace([np.inf, -np.inf], 0, inplace=True)
numeric_columns = df_s.select_dtypes(include='float').columns
df_s[numeric_columns] = df_s[numeric_columns].round(2)

Do you want to get the margin of the last month or specific month 
write the month like 'April'. Write anything else for the last 30 months for example n
n


>> We now have the **df_s** DataFrame containing the margins of our products

# **The Cleaning Part**

In [121]:
# 📅 Mapping of month names to their corresponding two-digit numbers
month_list = {
    'January':   '01',
    'February':  '02',
    'March':     '03',
    'April':     '04',
    'May':       '05',
    'June':      '06',
    'July':      '07',
    'August':    '08',
    'September': '09',
    'October':   '10',
    'November':  '11',
    'December':  '12'
}

# 🔷 Replace month names in the 'Month' column with two-digit numeric month codes
df['Month_num'] = df['Month'].replace(month_list)

# 🔷 Construct a date string column in the format DD-MM-YYYY (always using day '01').
# 🔷 We do this to ensure the data can be sorted by date, which will allow us to properly order the month columns in the report later.
df['Date'] = '01-' + df['Month_num'] + '-' + df['Year']

# 🔷 Convert the 'Date' column to actual pandas datetime objects
df['Date'] = pd.to_datetime(df['Date'])

# 🔷 Sort the DataFrame chronologically by the new 'Date' column
df.sort_values(by='Date', inplace=True)


In [122]:
# 🔷 Display the list of available countries to the user
print(list_countries)

# 🔷 Prompt the user to enter the country the user need to get its RSR report
inp_country = input('Enter the country you need: ')

# 🔷 Show the list of unique months available in the dataset
print(df['Month'].unique())

# 🔷 Prompt the user to enter the month they are interested in
inp_month = input('Enter the month you need (The first of the three months): ')

# 🔷 Filter both df, df_cp, and df_s DataFrames to include only rows for the selected country
df =df[df['Country'] == inp_country]
df_s =df_s[df_s['Country'] == inp_country]
df_cp =df_cp[df_cp['Country'] == inp_country]

# 🔷 Make a copy of the full country-specific DataFrame for further use if needed
df_all = df.copy()

# 🔷 Further filter df to include only rows for the selected month
df =df[df['Month'] == inp_month]

# 🔷 Getting the current year
current_year = datetime.now().year


# 🔷 Filter df and df_s to include only data for the current year
df =df[df['Year'] == str(current_year)]
df_s =df_s[df_s['Year'] == current_year]


['USA', 'UK', 'CA', 'EU', 'DE', 'FR', 'IT', 'ES', 'JP']
Enter the country you need: JP
['June' 'July' 'August' 'September' 'October' 'November' 'December'
 'January' 'February' 'March' 'April' 'May']
Enter the month you need (The first of the three months): May


>> At this point, both **df** and **df_all** contain only the data for the country selected by the user. **df** contains only the selected month, which we use as a reference to filter **df_all**, extracting the chosen month along with all subsequent months in chronological order. This ensures that **df_all** includes only the months needed for the analysis.


>> We also have **df_c** and **df_s**, which contain the prices and margins

In [123]:
# 🔷 Filter `df_all` to include only rows where the date is on or after the first date in the filtered `df`
# (i.e., keep data starting from the selected month and onward)
df_all = df_all[df_all['Date'] >= str(df['Date'].unique()[0].date())]

# 🔷 Extract the unique months present in the filtered `df_all`
list(df_all['Month'].unique())

# 🔷 Store the list of months in `order_month` for later use
# 🔷 The order of the months is very important in the report, so we need to have a list of months sorted by their date. This makes us sure that the order of months columns will be correct
order_month = list(df_all['Month'].unique())

In [124]:
# 🔷 Define the list of needed columns to keep from the original DataFrame
cols = [
    '(Child) Asin',
    'Session Percentage  Total',
    'Featured Offer (Buy Box) Percentage',
    'Units Ordered',
    'Unit Session Percentage',
    'Ordered Product Sales',
    'Month'
    ]

# 🔷 Select only these columns from `df` and rename '(Child) Asin' → 'ASIN'
df = df[cols].rename(columns = {'(Child) Asin': 'ASIN'})

# 🔷 Create a pivot table on `df`, aggregating metrics by ASIN
# Metrics are summed across all months for each ASIN
df = (
    pd.pivot_table(
        df,
        index=['ASIN'],
        values=[
            'Session Percentage  Total',
            'Featured Offer (Buy Box) Percentage',
            'Units Ordered',
            'Unit Session Percentage',
            'Ordered Product Sales'
        ],
        aggfunc='sum'
    )
    .reset_index()
)


# 🔷 Prepare `df_all` in the same way, keeping all data (not just filtered to one month)
df_all = df_all[cols]

# 🔷 Select only these columns from `df` and rename '(Child) Asin' → 'ASIN'
df_all = df_all[cols].rename(columns = {'(Child) Asin': 'ASIN'})


# 🔷 Create a pivot table on `df_all`, aggregating metrics by ASIN and Month
df_all = (
    pd.pivot_table(
        df_all,
        index=['ASIN', 'Month'],
        values=[
            'Session Percentage  Total',
            'Featured Offer (Buy Box) Percentage',
            'Units Ordered',
            'Unit Session Percentage',
            'Ordered Product Sales'
        ],
        aggfunc='sum'
    )
    .reset_index()
)




>> We have the ASINs of the products. Right now, we need to assign the names of this products based on the ASIN

>> We have a product list provided by Annies, titled “*List of All Products - NEW*”. **We will use this list to map ASINs to their corresponding product names.**

In [125]:
# 🔷 Define the root directory where the main product file is located
root_path = r"/content/drive/My Drive/Stock Products"

# 🔷 Change the current working directory to the root path
os.chdir(root_path)

# 🔷 Define the filename of the main Excel file
main_f = 'List of All Products - NEW.xlsx'

# 🔷 The file contains multiple sheets, each listing products for a specific country. We only need the sheet for the country selected by the user.
# 🔷 Try to load the sheet corresponding to the selected country
try:

    # Attempt to read the sheet named after the user-selected country
    df_main = pd.read_excel(main_f, sheet_name =inp_country )

    # Drop unnecessary columns for analysis
    df_main.drop(columns = ['SKU', 'Product Name', 'Brand',
       'Product link', 'Listing Status', 'Remarks'], inplace = True)

# 🔷 There isn’t a separate sheet for each EU country, so we read the products from the ‘EU’ sheet instead of a specific country sheet.
# 🔷 Fallback: if the selected country sheet (country in the EU) is not found, default to 'EU' sheet
except:

    # Attempt to read the EU sheet
    df_main = pd.read_excel(main_f, sheet_name ='EU' )

    # Drop the same unnecessary columns
    df_main.drop(columns = ['SKU', 'Product Name', 'Brand',
       'Product link', 'Listing Status', 'Remarks'], inplace = True)

>>  We now have the **df_main** DataFrame, which contains both the ASINs and their corresponding product names.

In [126]:
# 🔷 This report aims to track the performance of our top 60% of products, as the majority of our revenue comes from them.
# 🔷 Filter the DataFrame to keep only products whose 'Units Ordered' is above the 60th percentile (i.e., top 40% of products by units ordered)
df = df[df['Units Ordered'] > np.percentile(df['Units Ordered'], 60)].reset_index(drop = True)


# 🔷 Merge the filtered DataFrame (`df`) with the main product list (`df_main`)
# to bring in product names. Match on the 'ASIN' column.
# Keep only the first occurrence of each ASIN to avoid duplicates.
df = df.merge(df_main, how ='left', on = 'ASIN' ).drop_duplicates(subset = 'ASIN',keep = 'first')

In [127]:
# 🔷 Keep only the relevant columns for the report
df = df[['Product Name (Other versions)','ASIN', 'Session Percentage  Total',
       'Featured Offer (Buy Box) Percentage', 'Units Ordered',
       'Unit Session Percentage', 'Ordered Product Sales']]

# 🔷 Sort products by number of units ordered, in descending order, so the most important products appeared first
df.sort_values(by =  'Units Ordered', ascending  = False, inplace = True)

>> Now we have **df**, which contains all the necessary information for our report. However, the report needs to be structured in a specific format: with two layers of columns — the first layer representing the months, and under each month, the ASIN and metric columns.

In [128]:
# 🔷 Loop over each month in the desired order
for i in order_month:

    # Filter `df_all` to include only rows for the current month
    df_t = df_all[df_all['Month'] == i]

    # Select and order only the relevant columns for this month's data
    col = ['ASIN', 'Session Percentage  Total',
       'Featured Offer (Buy Box) Percentage', 'Units Ordered',
       'Unit Session Percentage', 'Ordered Product Sales']
    df_t = df_t[col]

    # Rename columns by appending the current month to each metric column name
    col = list(df_t.columns[1:] + ' '+i)  # Add month suffix to metrics
    col.append('ASIN')                    # Keep ASIN as is
    col = [col[-1]] + col[:-1]            # Ensure ASIN stays as the first column
    df_t.columns = col                    # Rename Columns

    # Merge this month's data into the main `df`, matching on ASIN
    df = df.merge(df_t, on = 'ASIN', how = 'left')

>> Previously, we used **df** as a template to merge month-specific data from **df_all**. This caused the original columns from the selected month in df to become duplicated and retain unclear names, so we drop them to clean up the result.

In [129]:
# 🔷 Define the list of junk columns that are not needed
cols = ['Session Percentage  Total',
       'Featured Offer (Buy Box) Percentage', 'Units Ordered',
       'Unit Session Percentage', 'Ordered Product Sales']

# 🔷 Drop these metric columns from `df`
df.drop(columns = cols, inplace = True)

# 🔷 Merge additional data (Net Margin and Price) from `df_s` into `df`, matching on ASIN
# If there are missing values after the merge, fill them with 'unavailable'
df = df.merge(df_s[['ASIN','Net Margin']], on ='ASIN', how = 'left').fillna('unavailable')
df = df.merge(df_cp[['ASIN','Price']], on ='ASIN', how = 'left').fillna('unavailable')

# Save Our Report In The Stock Products Folder

In [130]:
# 🔷 Define the root directory where the output file will be saved
root_path = r"/content/drive/My Drive/Stock Products"

# 🔷 Change the current working directory to the root path
os.chdir(os.path.join(root_path))

# 🔷 Save the final `df` DataFrame to an Excel file
# The file is named using the selected month and country (e.g., "January USA.xlsx")
df.to_excel(inp_month + ' ' + inp_country+ '.xlsx')